In [4]:
# (1) Importing dependency
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
np.random.seed(1000)

# AlexNet Architecture

In [8]:
# Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),\
 strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(23))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_20 (Activation)   (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_21 (Activation)   (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 8, 8, 256)         0         
__________

In [9]:
train_dir = 'Dataset/train'
test_dir = 'Dataset/final'
validation_dir = 'Dataset/test'

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True)

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(224, 224),
                    batch_size=32,
                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
                        validation_dir,
                        target_size=(224, 224),
                        batch_size=32,
                        class_mode='categorical')
datagen = ImageDataGenerator(rescale=1./255)
test=datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
       class_mode='categorical')

Found 35247 images belonging to 23 classes.
Found 8679 images belonging to 23 classes.
Found 1150 images belonging to 23 classes.


In [12]:

from keras import optimizers
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd,
                  metrics=['accuracy'])

In [13]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0,
                                  mode='min')


ckpt = ModelCheckpoint('alexNet_model.hdf5', save_best_only=True,
                           monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                       patience=7, verbose=1, epsilon=1e-4,
                                       mode='min')

In [14]:
history = model.fit_generator(
                train_generator,
                steps_per_epoch=100,
                epochs=50,
                callbacks=[earlyStopping, ckpt, reduce_lr_loss],
                validation_data=validation_generator,
                validation_steps=50)

Epoch 1/50
  2/100 [..............................] - ETA: 1205s - loss: 0.2191 - acc: 0.9538

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:

final=datagen.flow_from_directory(
        test_dir,
        target_size=(64, 64),
        batch_size=32,
       class_mode='categorical')

In [ ]:
#test model accurcy in new images 
test_accuracy=model.evaluate_generator(final,1150)

In [ ]:
print("Accuracy = ", test_accuracy[1]*100)